In [ ]:
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

def save_optimization_results(model_DA, model_RT, time_steps, nodes, filename='optimization_results.csv'):
    # Create a DataFrame to store the results
    da_results = []
    rt_results = []
    
    # Day-ahead results
    for k in range(time_steps):
        for i in range(nodes):
            da_results.append([
                'DA', k, i,
                model_DA.getVarByName(f'P_c[{i},{k}]').X,
                model_DA.getVarByName(f'R_U[{i},{k}]').X,
                model_DA.getVarByName(f'R_D[{i},{k}]').X,
                model_DA.getVarByName(f'delta_DA[{i},{k}]').X
            ])
    
    # Real-time results
    for k in range(time_steps):
        for i in range(nodes):
            rt_results.append([
                'RT', k, i,
                model_RT.getVarByName(f'r_U[{i},{k}]').X,
                model_RT.getVarByName(f'r_D[{i},{k}]').X,
                model_RT.getVarByName(f'P_lsh[{i},{k}]').X,
                model_RT.getVarByName(f'delta_RT[{i},{k}]').X,
                model_RT.getVarByName(f's[{i},{k}]').X
            ])
    
    # Convert to DataFrames
    da_columns = ['Stage', 'TimeStep', 'Node', 'P_c', 'R_U', 'R_D', 'delta_DA']
    rt_columns = ['Stage', 'TimeStep', 'Node', 'r_U', 'r_D', 'P_lsh', 'delta_RT', 's']
    
    df_da = pd.DataFrame(da_results, columns=da_columns)
    df_rt = pd.DataFrame(rt_results, columns=rt_columns)
    
    # Combine results
    df_results = pd.concat([df_da, df_rt], ignore_index=True)
    
    # Save to CSV
    df_results.to_csv(filename, index=False)


# Data allocation function
def allocate_data_to_nodes(data, ratio_bus):
    """
    Distribute a whole-system load dataset to 118 nodes based on ratio_bus.

    Args:
        data (np.array): Whole-system load dataset with shape (number of samples,)
        ratio_bus (np.array): Load distribution ratio with shape (118,)

    Returns:
        np.array: Distributed data with shape (number of samples, 118)
    """
    allocated_data = np.zeros((data.shape[0], 118))
    for i in range(data.shape[0]):
        allocated_data[i, :] = float(data[i]) * ratio_bus  # Convert data[i] to float
    return allocated_data

# Optimization model function
def calculate_optimization_cost_two_stage(P_l_predict, Y_real, topo, unit, ratio_bus, reserve_up=150, reserve_down=150, c=9000, c_s=1750):
    """
    Two-stage optimization model function. The input data is a single column and is distributed to 118 nodes based on ratio_bus.

    Args:
        P_l_predict (np.array): Predicted load values, shape (number of samples,)
        Y_real (np.array): Actual load values, shape (number of samples,)
        topo (pd.DataFrame): Topology data
        unit (pd.DataFrame): Generator unit parameter data
        ratio_bus (np.array): Load distribution ratio, shape (118,)
        reserve_up (int): Upward reserve requirement
        reserve_down (int): Downward reserve requirement
        c (int): Load shedding cost coefficient
        c_s (int): Slack variable cost coefficient

    Returns:
        float: Total optimization cost
    """
    # Distribute single-column data to 118 nodes
    P_l_predict_allocated = allocate_data_to_nodes(P_l_predict, ratio_bus)
    Y_real_allocated = allocate_data_to_nodes(Y_real, ratio_bus)
    P_l_predict_allocated = P_l_predict_allocated.T
    Y_real_allocated = Y_real_allocated.T

    nodes = topo.shape[0]
    time_steps = P_l_predict.shape[0]

    # ========== Day-ahead scheduling optimization problem ==========
    model_DA = gp.Model()

    # Define decision variables
    P_c = model_DA.addVars(nodes, time_steps, lb=0, name='P_c')  # Day-ahead unit generation
    R_U = model_DA.addVars(nodes, time_steps, lb=0, name='R_U')  # Upward reserve
    R_D = model_DA.addVars(nodes, time_steps, lb=0, name='R_D')  # Downward reserve
    delta_DA = model_DA.addVars(nodes, time_steps, lb=-180, ub=180, name='delta_DA')  # Day-ahead phase angle

    # Objective function
    obj_DA = gp.quicksum(
        R_U[i, j] * unit.iloc[i, 5] +
        R_D[i, j] * unit.iloc[i, 6] +
        P_c[i, j] * unit.iloc[i, 2]
        for i in range(nodes) for j in range(time_steps)
    )
    model_DA.setObjective(obj_DA, GRB.MINIMIZE)

    # Constraints
    for k in range(time_steps):
        # Reserve requirement constraints
        model_DA.addConstr(gp.quicksum(R_U[i, k] for i in range(nodes)) == reserve_up)
        model_DA.addConstr(gp.quicksum(R_D[i, k] for i in range(nodes)) == reserve_down)
        # Generator reserve capacity constraints
        for i in range(nodes):
            model_DA.addConstr(R_U[i, k] <= unit.iloc[i, 3])  # Upward reserve limit
            model_DA.addConstr(R_D[i, k] <= unit.iloc[i, 4])  # Downward reserve limit
        # Day-ahead market power balance
        for t in range(nodes):
            model_DA.addConstr(
                P_c[t, k] - P_l_predict_allocated[t, k] -
                gp.quicksum(topo.iloc[t, j] * (delta_DA[t, k] - delta_DA[j, k]) for j in range(nodes)) == 0
            )
        model_DA.addConstr(delta_DA[0, k] == 0)  # Reference node phase angle
        # Day-ahead unit generation constraints
        for i in range(nodes):
            model_DA.addConstr(P_c[i, k] <= unit.iloc[i, 1] - R_U[i, k])  # Maximum generation
            model_DA.addConstr(P_c[i, k] >= R_D[i, k])  # Minimum generation
        # Day-ahead line capacity constraints
        for i in range(nodes):
            for j in range(nodes):
                if i != j and topo.iloc[i, j] < -0.01:
                    model_DA.addConstr(topo.iloc[i, j] * (delta_DA[i, k] - delta_DA[j, k]) <= 175)
                    model_DA.addConstr(topo.iloc[i, j] * (delta_DA[i, k] - delta_DA[j, k]) >= -175)

    model_DA.optimize()

    # ========== Real-time scheduling optimization problem ==========
    model_RT = gp.Model()

    # Define decision variables
    r_U = model_RT.addVars(nodes, time_steps, lb=0, name='r_U')  # Actual upward adjustment
    r_D = model_RT.addVars(nodes, time_steps, lb=0, name='r_D')  # Actual downward adjustment
    P_lsh = model_RT.addVars(nodes, time_steps, lb=0, name='P_lsh')  # Load shedding
    delta_RT = model_RT.addVars(nodes, time_steps, lb=-180, ub=180, name='delta_RT')  # Real-time phase angle
    s = model_RT.addVars(nodes, time_steps, lb=0, name='s')  # Slack variable

    # Objective function
    obj_RT = gp.quicksum(
        (r_U[i, j] - r_D[i, j]) * unit.iloc[i, 2]
        for i in range(nodes) for j in range(time_steps)
    )
    obj_RT += gp.quicksum(P_lsh[i, j] * c for i in range(nodes) for j in range(time_steps))
    obj_RT += gp.quicksum(s[i, j] * c_s for i in range(nodes) for j in range(time_steps))
    model_RT.setObjective(obj_RT, GRB.MINIMIZE)

    # Constraints
    for k in range(time_steps):
        # Real-time market power balance
        for t in range(nodes):
            model_RT.addConstr(
                r_U[t, k] - r_D[t, k] - s[t, k] - Y_real_allocated[t, k] + P_l_predict_allocated[t, k] + P_lsh[t, k] -
                gp.quicksum(
                    topo.iloc[t, j] * (delta_RT[t, k] - delta_RT[j, k]) -
                    topo.iloc[t, j] * (delta_DA[t, k].X - delta_DA[j, k].X)
                    for j in range(nodes)
                ) == 0
            )
        # Real-time reserve adjustment constraints
        for i in range(nodes):
            model_RT.addConstr(r_U[i, k] <= R_U[i, k].X)  # Actual upward adjustment must not exceed reserve
            model_RT.addConstr(r_D[i, k] <= R_D[i, k].X)  # Actual downward adjustment must not exceed reserve
        model_RT.addConstr(delta_RT[0, k] == 0)  # Reference node phase angle
        # Real-time line capacity constraints
        for i in range(nodes):
            for j in range(nodes):
                if i != j and topo.iloc[i, j] < -0.01:
                    model_RT.addConstr(topo.iloc[i, j] * (delta_RT[i, k] - delta_RT[j, k]) <= 175)
                    model_RT.addConstr(topo.iloc[i, j] * (delta_RT[i, k] - delta_RT[j, k]) >= -175)

    model_RT.optimize()

    # Calculate total cost
    total_cost = model_DA.ObjVal + model_RT.ObjVal

    # Print two-stage cost values and total cost
    print(f"Total cost: {total_cost}")

    # Call the function to save results
    save_optimization_results(model_DA, model_RT, time_steps, nodes, 'optimization_results.csv')

    return total_cost


# Read Y_test.csv file and remove the first-row column name
Y_real = pd.read_csv('Y_test.csv').values.flatten()
P_l_predict = pd.read_csv('prediction_GB.csv').values.flatten()

# System parameters (topology data and generator unit data)
topo = pd.read_excel('118nodes_system.xlsx', sheet_name='topology', index_col=None, header=None) 
unit = pd.read_excel('118nodes_system.xlsx', sheet_name='unit')

# mpc_bus data
mpc_bus = [
    [1, 51], [2, 20], [3, 39], [4, 39], [5, 0], [6, 52], [7, 19], [8, 28], [9, 0], [10, 0],
    [11, 70], [12, 47], [13, 34], [14, 14], [15, 90], [16, 25], [17, 11], [18, 60], [19, 45], [20, 18],
    [21, 14], [22, 10], [23, 7], [24, 13], [25, 0], [26, 0], [27, 71], [28, 17], [29, 24], [30, 0],
    [31, 43], [32, 59], [33, 23], [34, 59], [35, 33], [36, 31], [37, 0], [38, 0], [39, 27], [40, 66],
    [41, 37], [42, 96], [43, 18], [44, 16], [45, 53], [46, 28], [47, 34], [48, 20], [49, 87], [50, 17],
    [51, 17], [52, 18], [53, 23], [54, 113], [55, 63], [56, 84], [57, 12], [58, 12], [59, 277], [60, 78],
    [61, 0], [62, 77], [63, 0], [64, 0], [65, 0], [66, 39], [67, 28], [68, 0], [69, 0], [70, 66],
    [71, 0], [72, 12], [73, 6], [74, 68], [75, 47], [76, 68], [77, 61], [78, 71], [79, 39], [80, 130],
    [81, 0], [82, 54], [83, 20], [84, 11], [85, 24], [86, 21], [87, 0], [88, 48], [89, 0], [90, 163],
    [91, 10], [92, 65], [93, 12], [94, 30], [95, 42], [96, 38], [97, 15], [98, 34], [99, 42], [100, 37],
    [101, 22], [102, 5], [103, 23], [104, 38], [105, 31], [106, 43], [107, 50], [108, 2], [109, 8], [110, 39],
    [111, 0], [112, 68], [113, 6], [114, 8], [115, 22], [116, 184], [117, 20], [118, 33]
]

# Calculate load distribution ratio
ratio_bus = np.zeros(118)
for i in range(118):
    ratio_bus[i] = mpc_bus[i][1] / sum(mpc_bus[j][1] for j in range(118))

# Call the optimization model
total_cost = calculate_optimization_cost_two_stage(P_l_predict, Y_real, topo, unit, ratio_bus)


Academic license - for non-commercial use only - expires 2026-03-30
Using license file C:\Users\85033\gurobi.lic
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 10 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 714714 rows, 257712 columns and 1622166 nonzeros
Model fingerprint: 0xc6e53ec7
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [4e+01, 2e+02]
  Bounds range     [2e+02, 2e+02]
  RHS range        [3e-01, 8e+02]

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve removed 578214 rows and 159978 columns
Presolve time: 1.29s
Presolved: 136500 rows, 164346 columns, 617869 nonzeros

Ordering time: 0.06s

Barrier statistics:
 AA' NZ     : 1.743e+06
 Factor NZ  : 3.103e+06 (roughly 150 MBytes of memory)
 Factor Ops : 7.559e+07 (less than 1 second per iteration)
 Threads    : 8

                  Objective                Residual
Iter       Primal   